In [1]:
import pandas as pd

In [39]:
df = pd.read_csv('data_faq_mfc.csv')

In [3]:
from deeppavlov.dataset_readers import faq_reader
from deeppavlov.core.common.chainer import Chainer
from deeppavlov.core.data.data_learning_iterator import DataLearningIterator
from deeppavlov import configs, train_model
# faq_reader(data_path='data_faq_mfc.csv', x_col_name='Question', y_col_name='Answer')

In [ ]:
import json 

In [15]:


# config = '''
# {
#   "dataset_reader": {
#     "class_name": "faq_reader",
#     "x_col_name": "Question",
#     "y_col_name": "Answer",
#     "data_path": "data_faq_mfc.csv"
#   },
#   "dataset_iterator": {
#     "class_name": "data_learning_iterator",
#     "field_to_split": "train",
#     "split_fields": ["train","test"],
#     "split_proportions": [0.8,0.2]
#   },
#   "chainer": {
#     "in": "question",
#     "pipe": [
#       {
#         "class_name": "ru_tokenizer",
#         "in": "question",
#         "lemmas": true,
#         "out": "q_token_lemmas"
#       },
#       {
#         "class_name": "fasttext",
#         "in": "q_token_lemmas",
#         "load_path": "{DOWNLOADS_PATH}/embeddings/lenta_lower_100.bin",
#         "mean": true,
#         "out": "question_vector"
#       },
#       {
#         "class_name": "cos_sim_classifier",
#         "in": "question_vector",
#         "fit_on": [
#           "question_vector",
#           "y"
#         ],
#         "top_n": 3,
#         "save_path": "{MODELS_PATH}/faq/fasttext_cos_classifier.pkl",
#         "load_path": "{MODELS_PATH}/faq/fasttext_cos_classifier.pkl",
#         "out": [
#           "answer",
#           "score"
#         ]
#       }
#     ],
#     "out": [
#       "answer",
#       "score"
#     ]
#   },
#   "train": {
#     "evaluation_targets": [],
#     "class_name": "fit_trainer",
#      "metrics": ["accuracy"]
#   },
#   "metadata": {
#     "variables": {
#       "ROOT_PATH": "/home/user/src/deeppavlov",
#       "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
#       "MODELS_PATH": "{ROOT_PATH}/models"
#     },
#     "download": [
#       {
#         "url": "http://files.deeppavlov.ai/faq/school/fasttext_cos_classifier.pkl",
#         "subdir": "{MODELS_PATH}/faq"
#       },
#       {
#         "url": "http://files.deeppavlov.ai/embeddings/lenta_lower_100.bin",
#         "subdir": "{DOWNLOADS_PATH}/embeddings"
#       }
#     ]
#   }
# }
# '''

# config = json.loads(config)

In [10]:
config = '''
{
  "dataset_reader": {
    "class_name": "faq_reader",
    "x_col_name": "Question",
    "y_col_name": "Answer",
    "data_path": "data_faq_mfc.csv"
  },
  "dataset_iterator": {
    "class_name": "basic_classification_iterator",
    "seed": 42
  },
  "chainer": {
    "in": "q",
    "in_y": "y",
    "pipe": [
      {
        "class_name": "ru_tokenizer",
        "id": "my_tokenizer",
        "in": "q",
        "lemmas": true,
        "out": "q_token_lemmas"
      },
      {
        "ref": "my_tokenizer",
        "in": "q_token_lemmas",
        "out": "q_lem"
      },
      {
        "in": [
          "q_lem"
        ],
        "out": [
          "q_vect"
        ],
        "fit_on": [
          "q_lem"
        ],
        "id": "tfidf_vec",
        "class_name": "sklearn_component",
        "save_path": "{MODELS_PATH}/vectorizer/tfidf_vectorizer_ruwiki_v2.pkl",
        "load_path": "{MODELS_PATH}/vectorizer/tfidf_vectorizer_ruwiki_v2.pkl",
        "model_class": "sklearn.feature_extraction.text:TfidfVectorizer",
        "infer_method": "transform"
      },
      {
        "id": "answers_vocab",
        "class_name": "simple_vocab",
        "fit_on": [
          "y"
        ],
        "save_path": "{MODELS_PATH}/faq/ru_mipt_answers.dict",
        "load_path": "{MODELS_PATH}/faq/ru_mipt_answers.dict",
        "in": "y",
        "out": "y_ids"
      },
      {
        "in": "q_vect",
        "fit_on": [
          "q_vect",
          "y_ids"
        ],
        "out": [
          "y_pred_proba"
        ],
        "class_name": "sklearn_component",
        "main": true,
        "save_path": "{MODELS_PATH}/faq/tfidf_logreg_classifier_v2.pkl",
        "load_path": "{MODELS_PATH}/faq/tfidf_logreg_classifier_v2.pkl",
        "model_class": "sklearn.linear_model:LogisticRegression",
        "infer_method": "predict_proba",
        "C": 1000,
        "penalty": "l2"
      },
      {
        "in": "y_pred_proba",
        "out": "y_pred_ids",
        "class_name": "proba2labels",
        "max_proba": true
      },
      {
        "in": "y_pred_ids",
        "out": "y_pred_answers",
        "ref": "answers_vocab"
      }
    ],
    "out": [
      "y_pred_answers",
      "y_pred_proba"
    ]
  },
  "train": {
    "class_name": "fit_trainer",
    "evaluation_targets": []
  },
  "metadata": {
    "variables": {
      "ROOT_PATH": "/home/user/src/deeppavlov",
      "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
      "MODELS_PATH": "{ROOT_PATH}/models"
    },
    "download": [
      {
        "url": "http://files.deeppavlov.ai/faq/school/tfidf_logreg_classifier_v2.pkl",
        "subdir": "{MODELS_PATH}/faq"
      },
      {
        "url": "http://files.deeppavlov.ai/vectorizer/tfidf_vectorizer_ruwiki_v2.pkl",
        "subdir": "{MODELS_PATH}/vectorizer"
      },
      {
        "url": "http://files.deeppavlov.ai/faq/mipt/ru_mipt_answers.dict",
        "subdir": "{MODELS_PATH}/faq"
      }
    ]
  }
}
'''

config = json.loads(config)

In [14]:
config

{'dataset_reader': {'class_name': 'faq_reader',
  'x_col_name': 'Question',
  'y_col_name': 'Answer',
  'data_path': 'data_faq_mfc.csv'},
 'dataset_iterator': {'class_name': 'basic_classification_iterator',
  'seed': 42},
 'chainer': {'in': 'q',
  'in_y': 'y',
  'pipe': [{'class_name': 'ru_tokenizer',
    'id': 'my_tokenizer',
    'in': 'q',
    'lemmas': True,
    'out': 'q_token_lemmas'},
   {'ref': 'my_tokenizer', 'in': 'q_token_lemmas', 'out': 'q_lem'},
   {'in': ['q_lem'],
    'out': ['q_vect'],
    'fit_on': ['q_lem'],
    'id': 'tfidf_vec',
    'class_name': 'sklearn_component',
    'save_path': '{MODELS_PATH}/vectorizer/tfidf_vectorizer_ruwiki_v2.pkl',
    'load_path': '{MODELS_PATH}/vectorizer/tfidf_vectorizer_ruwiki_v2.pkl',
    'model_class': 'sklearn.feature_extraction.text:TfidfVectorizer',
    'infer_method': 'transform'},
   {'id': 'answers_vocab',
    'class_name': 'simple_vocab',
    'fit_on': ['y'],
    'save_path': '{MODELS_PATH}/faq/ru_mipt_answers.dict',
    'load_

In [29]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

In [66]:
model_conf = read_json(configs.faq.tfidf_autofaq)
model_conf['dataset_reader']['data_path'] = "data_faq_mfc.csv"
model_conf['dataset_reader']['data_url'] = "data_faq_mfc.csv"

faq = train_model(model_conf)

2022-01-21 01:31:28.13 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 203: Loading model sklearn.feature_extraction.text:TfidfVectorizer from /home/user/.deeppavlov/models/vectorizer/tfidf_vectorizer_ruwiki.pkl
2022-01-21 01:31:28.17 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 210: Model sklearn.feature_extraction.textTfidfVectorizer loaded  with parameters
2022-01-21 01:31:28.18 WARNING in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 216: Fitting of loaded model can not be continued. Model can be fitted from scratch.If one needs to continue fitting, please, look at `warm_start` parameter
2022-01-21 01:31:28.817 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 109: Fitting model sklearn.feature_extraction.textTfidfVectorizer
2022-01-21 01:31:28.845 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 241: Saving model t

In [31]:
# read_json(model_conf)

In [12]:
# !sudo apt install libpython3.7-dev
# !pip install fasttext

In [67]:
faq(["У меня нет прописки"])

[['Здравствуйте! Для более точного ответа просим Вас обратиться в региональный call-центр МФЦ по номеру телефона 8(4162)99-22-22 так как по Вашему запросу необходима дополнительная информация.'],
 [0.02]]

In [50]:
df.iloc[22, :].values

array(['Что означает :"Передано на выдачу в ОГВ"?',
       'Здравствуйте! Просим Вас уточнить номер дела или описи указанный в расписке выданной в МФЦ. Также о статусе рассмотрения дела Вы можете узнать позвонив в региональный call-центр МФЦ по номеру телефона 8 (4162) 99-22-22. В случае если документы подавались Вами через Единый портал государственных услуг о готовности результата Вы можете уточнить в Органе предоставляющем услугу.'],
      dtype=object)

In [61]:
model_conf = read_json(configs.faq.fasttext_avg_autofaq)
model_conf['dataset_reader']['data_path'] = "data_faq_mfc.csv"
model_conf['dataset_reader']['data_url'] = None
model_conf['metadata']['variables']['ROOT_PATH'] = '/home/user/src/deeppavlov'

In [62]:
faq = train_model(model_conf)

2022-01-21 01:30:16.779 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 53: [loading fastText embeddings from `/home/user/src/deeppavlov/downloads/embeddings/lenta_lower_100.bin`]
2022-01-21 01:30:20.46 INFO in 'deeppavlov.models.classifiers.cos_sim_classifier'['cos_sim_classifier'] at line 128: Saving faq_model to /home/user/src/deeppavlov/models/faq/fasttext_cos_classifier.pkl
2022-01-21 01:30:20.188 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 53: [loading fastText embeddings from `/home/user/src/deeppavlov/downloads/embeddings/lenta_lower_100.bin`]
2022-01-21 01:30:22.483 INFO in 'deeppavlov.models.classifiers.cos_sim_classifier'['cos_sim_classifier'] at line 133: Loading faq_model from /home/user/src/deeppavlov/models/faq/fasttext_cos_classifier.pkl
2022-01-21 01:30:22.629 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 53: [loading fastText embeddings from `/home/user

In [65]:
faq(["У меня нет пропискии"])

[['Добрый день! Список документов для оформления паспорта после исполнения 14 лет включает:\n- Заявление (заполняется в МФЦ);\n- Оригинал свидетельства о рождении;\n- Документ подтверждающий регистрацию по месту жительства (необязательный);\n- Две фотографии\xa0\xa0(35x45 мм);\n- Квитанция об оплате государственной пошлины (300 руб оплатить можно в МФЦ).\nДокументы подтверждающие гражданство Российской Федерации определены п.п. «а» «б» «В» п.1 Указа Президента Российской Федерации №444 от 13.04.2011 года:\n1.Свидетельство о рождении в котором имеются сведения о гражданстве Российской Федерации (одно из перечисленных условий):\n- обоих родителей или единственного родителя;\n- одного из родителей если другой родитель является лицом без гражданства или признан безвестно отсутствующим либо если место его нахождения неизвестно;\n- одного из родителей и гражданстве иностранного государства другого родителя (если свидетельство о рождении выдано на территории Российской Федерации);\n 2. Действ